# Student self-graded homework workflow
Author: Dr. Matthew Ford (mattford@uw.edu) [Website](https://dashdotrobot.com)

M. Ford and H. E. Dillon, __[A secure, scalable approach to student-graded homework for self-reflection](https://peer.asee.org/a-secure-scalable-approach-to-student-graded-homework-for-self-reflection)__. 2024 ASEE Annual Conference & Exposition, Portland, Oregon. 10.18260/1-2--46489

Use this script to compare student self-graded scores to instructor scores and enter final grades into the Canvas gradebook.

### Workflow for a student self-graded homework assignment

1. __Assign homework on Canvas__: Create a zero-point assignment called "Homework X - submit to Gradescope". This assignment should have the problem set, instructions, etc.
2. __Create Gradescope assignment__: Create a Gradescope assignment with the correct number of problems. Each problem should have a point value of 0. Students will actually submit their assignment here.
3. __Create a solution and grading rubric__: The grading rubric should be clear enough for students to follow. I make my rubric into a Word Doc that students fill out when they grade their work. Upload the solution and rubric to Canvas, but set them to be unavailable until after the homework deadline.
5. __Create a Canvas Quiz for self-grading__: Create a Canvas Quiz ("Classic Quiz" or "Legacy Quiz") called "Homework X - enter your own grade". I put a Text Only question at the top of the quiz with links to the solution and rubric. The quiz should have a Numeric question for each homework problem. The name of each problem should be "Problem Y" with the problem number and the text should be "Please enter your score for Problem Y." The point value for the question should match the point value for the homework problem. Since the quiz score itself is irrelevant, you can make a single correct answer "-1".
6. __Set the self-grading quiz "Available From" time__: Set the quiz to become available just after the homework deadline. I suggest a 2 hour gap or so to account for late submissions.

After students submit their homework, the self-grading assignment becomes available and students are able to review the solutions and enter their scores into the Canvas quiz. The instructor or TAs should pick a "check problem" to grade via Gradescope and compare to students' self-assessments.

7. __Grade check problem on Gradescope__: Change the point value of the check problem on Gradescope to match the actual points and create a rubric. After you're done grading, sync the Gradescope score back to the Canvas assignment "Homework X - submit to Gradescope".
8. __Reconcile and sync scores__: After you and the students have submitted your grades, run this script to compare scores. This script will generate a log file which you can check for consistency. You may want to look more carefully at submissions with a large discrepancy between the instructor score and student score. Finally, change `enter_into_gradebook` to `True` and run this script again to publish final scores to Canvas. You may have to open Canvas to publish the assignment "Homework X - final grade"

In [ ]:
from canvasapi import Canvas
from datetime import datetime
import pandas as pd
from numpy import argmax
import csv

## Canvas course settings

Enter the number after https://canvas.[institution].edu/courses/ in your course URL.

In [ ]:
course_id = 0

In [ ]:
with open('API_KEY.txt') as f:
    API_KEY = f.readline().strip()

canvas = Canvas('https://canvas.uw.edu', API_KEY)
course = canvas.get_course(course_id)

## Assignment settings

In [ ]:
# Assignment name
assignment_name = "Homework 1"

# Name of the problem to be checked against the instructor assigned score
check_problem = 'Problem 1'

# Name of self-grading Canvas quiz
self_grade_name = assignment_name + " - enter your own grade"

# Name of Canvas assignment for syncing Gradescope scores for one problem
inst_grade_name = assignment_name + " - submit to Gradescope"

# Name of Canvas assignment to store final assignment score (the script will create this automaticaly)
final_grade_name = assignment_name + " - final grade"

# Filename for the status log file
log_filename = assignment_name + "_log.csv"

## Update settings

In [ ]:
enter_into_gradebook = True   # Set to False to generate a report without entering grades
replace_student_score = True  # Set to False to default to student scores for check problem

## Reconcile grades

Compare student score to instructor score for check problem and generate a log file.

In [ ]:
# Fetch self-grading assignment
a_self = course.get_assignments(search_term=self_grade_name)[0]
a_inst = course.get_assignments(search_term=inst_grade_name)[0]

# Get point values of quiz questions corresponding to Problems
q_probs = course.get_quiz(a_self.quiz_id).get_questions()
q_point_vals = {q.id:q.points_possible for q in q_probs if 'Problem' in q.question_name}
q_names = {q.id:q.question_name for q in q_probs if 'Problem' in q.question_name}

# Calculate total assignment point value
total_point_val = sum(q_point_vals.values())

# Fetch or create final grade assignment
try:
    a_final = course.get_assignments(search_term=final_grade_name)[0]
    print(final_grade_name, 'already exists.')
except IndexError as e:
    print(final_grade_name, 'does not exist. Creating new assignment...')
    a_final = course.create_assignment({
        'name': final_grade_name,
        'points_possible': total_point_val
    })

# Log file columns:
cols = ['user_id', 'name', 'total_score', 'self_score', 'inst_score', 'diff', 'notes']

with open(log_filename, 'w', newline='') as log_file:
    log = csv.DictWriter(log_file, fieldnames=cols)
    log.writeheader()

    for user in course.get_users(enrollment_type=['student']):

        try:

            # Get self grade submission history (includes all attempts)
            s = a_self.get_submission(user=user.id, include='submission_history')

            # Get latest submission
            sh_index = argmax([sh['attempt'] for sh in s.submission_history])
            sh_latest = s.submission_history[sh_index]

            if 'submission_data' not in sh_latest:
                raise KeyError('No student self grade found')

            # Get instructor score
            x = a_inst.get_submission(user=user.id)

            if x.grade is None:
                raise KeyError('No instructor grade found for student')

            inst_check_score = float(x.grade)
            self_check_score = 0
            total_score = 0

            for q in sh_latest['submission_data']:    # Loop over quiz questions
                if q['question_id'] in q_point_vals:  # If Problem
                    student_score = float(q['text'])
                    
                    if student_score > q_point_vals[q['question_id']]:
                        raise ValueError('Student entered score greater than maximum.')
                        
                    if student_score < 0:
                        raise ValueError('Student entered a score less than zero.')

                    if q_names[q['question_id']] == check_problem:
                        self_check_score = student_score
                        if replace_student_score:
                            total_score += inst_check_score
                        else:
                            total_score += student_score
                    else:
                        total_score += student_score

            # Update score in Canvas
            if enter_into_gradebook:
                s_final = a_final.get_submission(user.id)
                s_final.edit(submission={'posted_grade':total_score})
                        
            # Log score info
            log.writerow({'user_id': user.id,
                          'name': user.name,
                          'total_score': total_score,
                          'self_score': self_check_score,
                          'inst_score': inst_check_score,
                          'diff': inst_check_score - self_check_score})

        except Exception as e:  # An error occurred
            print(repr(e))
            log.writerow({'user_id': user.id, 'name': user.name, 'notes': repr(e)})
            
print('Done')